In [16]:
import os

os.environ['WANDB_NOTEBOOK_NAME'] = '/Users/aditya/Programming/stock-market-predictions/stock_market_predictions/forecasting/GRU-5-Year/MSFT-5-Year-GRU.ipynb'

In [17]:
def train():
    import pandas as pd
    from os.path import abspath

    import warnings
    warnings.filterwarnings("ignore")
    df = pd.read_csv(abspath('../../../data/MSFT/MSFT-5-Year.csv'), index_col=0, parse_dates=['Date'])

    df.columns.name = df.index.name
    df.index.name = None

    df = df.filter(['Date', 'Open'])
    # Split the df into a training set and a test set
    from sklearn.model_selection import train_test_split

    # Split the data into training and testing sets
    train_set, test_set = train_test_split(df, test_size=0.05, shuffle=False)

    n_features = 1
    from sklearn.preprocessing import MinMaxScaler

    transformer = MinMaxScaler()
    train_scaled = transformer.fit_transform(train_set)
    test_scaled = transformer.fit_transform(test_set)
    
    import tensorflow as tf
    from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator # type: ignore

    n_input = 60

    train_generator=TimeseriesGenerator(train_scaled,
                                        train_scaled,
                                        length=n_input,
                                        batch_size=32)
    
    from keras.models import Sequential
    from keras.layers import Dense, GRU

    GRU_UNITS = 20

    model = Sequential()
    model.add(GRU(units=GRU_UNITS, input_shape=(n_input, n_features)))
    model.add(Dense(units=1))
    
    # Create an EarlyStopping callback
    from keras.callbacks import EarlyStopping
    early_stopping = EarlyStopping(monitor='loss', patience=5)
    
    import wandb
    from wandb.keras import WandbCallback

    wandb.init(project="stock-predictions", entity="alpheron", magic=True)

    wandb.config = {
    "learning_rate": 0.001,
    "epochs": 100,
    "batch_size": 32,
    "lstm_units": GRU_UNITS
    }

    from keras.metrics import RootMeanSquaredError
    from tensorflow_addons.optimizers import RectifiedAdam

    model.compile(optimizer=RectifiedAdam(), loss='mean_squared_error', metrics=[RootMeanSquaredError()])
    model.fit(train_generator, epochs=100, callbacks=[early_stopping, WandbCallback()])
    model.save(abspath('../../../logs/MSFT-5-Year/'))

    inputs = df[len(df)-len(test_set)-60 : ].values
    inputs = inputs.reshape(-1,1)
    inputs = transformer.transform(inputs)
    import numpy as np

    x_test = []
    for i in range(60,len(inputs)):
        x_test.append(inputs[i-60:i, 0])
        
    x_test = np.array(x_test)
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
    print(x_test.shape)
    predicted = model.predict(x_test)
    predicted = transformer.inverse_transform(predicted)

    # Convert the predicted values to a dataframe
    predicted_df = pd.DataFrame(predicted)

    # Rename the column to "Open"
    predicted_df.columns = ['Open']

    # Add a new column called "Date" that has the same index as the other dataframe
    predicted_df['Date'] = df[len(df)-len(test_set)-0 : ].index

    # Set the 'Date' column as the index
    predicted_df = predicted_df.set_index('Date')

    import math
    from sklearn.metrics import mean_squared_error

    def return_rmse(test, predicted):
        rmse = math.sqrt(mean_squared_error(test, predicted))
        print("the root mean squared error is: {}".format(rmse))
        return rmse

    RMSE = return_rmse(test_set, predicted)
    wandb.log({"root_mean_square_error" : RMSE})
    artifact = wandb.Artifact('Time-Series', type='model')
    artifact.add_dir(abspath('../../../logs/MSFT-5-Year/'))
    wandb.run.log_artifact(artifact)

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/alpheron/stock-predictions/3mjmhs28/file_stream
NoneType: None


In [18]:
sweep_config = {
    'method': 'bayes',
    'metric': 
        {
            'name': 'root_mean_square_error',
            'goal': 'minimize',
        },
    'parameters': {
        'GRU_UNITS': {
            'values': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 120, 125, 130, 135, 140, 145, 150]
        },
        'epochs': {
            'value': 100
        }
    },
    'early_terminate': {
        'type': 'hyperband',
        's': 2,
        'eta': 3,
        'max_iter': 50
    }
}

In [19]:
import wandb
sweep_id = wandb.sweep(sweep_config, project="stock-predictions", entity="alpheron")

Create sweep with ID: 831500mq
Sweep URL: https://wandb.ai/alpheron/stock-predictions/sweeps/831500mq


In [20]:
wandb.agent(sweep_id, function=train, project="stock-predictions", entity="alpheron")

wandb: Agent Starting Run: i768whn4 with config:
wandb: 	GRU_UNITS: 150
wandb: 	epochs: 100
wandb: Currently logged in as: alpheron (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch 1/100


2022-02-27 13:58:30.076839: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/alpheron/stock-predictions/ys0nc3xp/file_stream
NoneType: None
2022-02-27 13:58:31.055303: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-02-27 13:58:32.284733: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


36/36 [==============================] - 16s 218ms/step - loss: 0.2577 - root_mean_squared_error: 0.5076
Epoch 2/100
36/36 [==============================] - 6s 180ms/step - loss: 0.2066 - root_mean_squared_error: 0.4545
Epoch 3/100
36/36 [==============================] - 6s 163ms/step - loss: 0.1453 - root_mean_squared_error: 0.3812
Epoch 4/100
36/36 [==============================] - 6s 157ms/step - loss: 0.0956 - root_mean_squared_error: 0.3092
Epoch 5/100
10/36 [=======>......................] - ETA: 3s - loss: 0.0846 - root_mean_squared_error: 0.2908

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/alpheron/stock-predictions/ys0nc3xp/file_stream
NoneType: None


...

36/36 [==============================] - 5s 147ms/step - loss: 0.0171 - root_mean_squared_error: 0.1308
Epoch 13/100
36/36 [==============================] - 6s 166ms/step - loss: 0.0135 - root_mean_squared_error: 0.1161
Epoch 14/100
36/36 [==============================] - 6s 174ms/step - loss: 0.0095 - root_mean_squared_error: 0.0975
Epoch 15/100
36/36 [==============================] - 6s 160ms/step - loss: 0.0060 - root_mean_squared_error: 0.0775
Epoch 16/100
 5/36 [===>..........................] - ETA: 4s - loss: 0.0043 - root_mean_squared_error: 0.0655

wandb: Ctrl + C detected. Stopping sweep.


wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/alpheron/stock-predictions/ys0nc3xp/file_stream
NoneType: None


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▇▅▄▃▂▂▂▂▂▁▁▁▁▁
root_mean_squared_error,█▇▆▅▄▃▃▃▃▂▂▂▂▁▁
epoch,14
loss,0.006
root_mean_squared_error,0.07746


wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/alpheron/stock-predictions/3mjmhs28/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/alpheron/stock-predictions/ys0nc3xp/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/alpheron/stock-predictions/ys0nc3xp/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/alpheron/stock-predictions/ys0nc3xp/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 40